In [1]:
import math
import os
import sys
from typing import Iterable
import numpy as np
import copy
import itertools
import argparse

import torch

import util.misc as utils
from datasets.coco_eval import CocoEvaluator
from datasets.panoptic_eval import PanopticEvaluator
from datasets.hico_eval import HICOEvaluator
from datasets.vcoco_eval import VCOCOEvaluator

In [2]:
parser = argparse.ArgumentParser('Set transformer detector', add_help=False)
parser.add_argument('--dataset_file', default='coco')
parser.add_argument('--hoi_path', type=str)
parser.add_argument('--num_queries', default=100, type=int,
                        help="Number of query slots")
args = parser.parse_args(args=[])

args.dataset_file = 'vcoco'
args.hoi_path = '/hub_data3/intern/seokwon/qpic/data/v-coco'
args.num_queries = 100
args

Namespace(dataset_file='vcoco', hoi_path='/hub_data3/intern/seokwon/qpic/data/v-coco', num_queries=100)

In [3]:
from datasets import build_dataset
from torch.utils.data import DataLoader

dataset_train = build_dataset(image_set='train', args=args)
train_anno = dataset_train.annotations

In [4]:
object_label = [
    'no_object', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', \
    'traffic light', 'fire hydrant', 'street sign', 'stop sign', 'parking meter', 'bench', 'bird', \
    'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'hat', 'backpack', \
    'umbrella', 'shoe', 'eye glasses', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', \
    'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', \
    'bottle', 'plate', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', \
    'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', \
    'potted plant', 'bed', 'mirror', 'dining table', 'window', 'desk', 'toilet', 'door', 'tv', 'laptop', \
    'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', \
    'blender', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush', 'hair brush']

verb_label = [
    'hold_obj', 'stand', 'sit_instr', 'ride_instr', 'walk', 'look_obj', 'hit_instr', 'hit_obj', \
    'eat_obj', 'eat_instr', 'jump_instr', 'lay_instr', 'talk_on_phone_instr', 'carry_obj', \
    'throw_obj', 'catch_obj', 'cut_instr', 'cut_obj', 'run', 'work_on_computer_instr', \
    'ski_instr', 'surf_instr', 'skateboard_instr', 'smile', 'drink_instr', 'kick_obj', \
    'point_instr', 'read_obj', 'snowboard_instr']

In [5]:
from itertools import permutations

hois_dict = dict()

for sample in train_anno:
    objs = sample['annotations']
    hois = sample['hoi_annotation']
    
    hoi_dict = dict()
    for hoi in hois:
        subject_class = hoi['subject_id']
        object_class  = hoi['object_id']
        hoi_class     = hoi['category_id']
        
        if subject_class not in hoi_dict:
            hoi_dict[subject_class] = dict()
        if object_class not in hoi_dict[subject_class]:
            hoi_dict[subject_class][object_class] = set()
        hoi_dict[subject_class][object_class].add(hoi_class)
        
    for sbj in hoi_dict.keys():
        for obj in hoi_dict[sbj].keys():
            d = hoi_dict[sbj][obj]
            
            # real label for subject and object
            subject_id = objs[sbj]['category_id']
            object_id = objs[obj]['category_id']
            
            for verb_comb in list(itertools.product(list(d), repeat=2)):
                if subject_id not in hois_dict:
                    hois_dict[subject_id] = dict()
                if object_id not in hois_dict[subject_id]:
                    hois_dict[subject_id][object_id] = dict()
                if verb_comb[0] not in hois_dict[subject_id][object_id]:
                    hois_dict[subject_id][object_id][verb_comb[0]] = dict()
                if verb_comb[1] not in hois_dict[subject_id][object_id][verb_comb[0]]:
                    hois_dict[subject_id][object_id][verb_comb[0]][verb_comb[1]] = 1
                else:
                    hois_dict[subject_id][object_id][verb_comb[0]][verb_comb[1]] += 1
                

In [6]:
a = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13,
   14, 15, 16, 17, 18, 19, 20, 21, 22, 23,
   24, 25, 27, 28, 31, 32, 33, 34, 35, 36,
   37, 38, 39, 40, 41, 42, 43, 44, 46, 47,
   48, 49, 50, 51, 52, 53, 54, 55, 56, 57,
   58, 59, 60, 61, 62, 63, 64, 65, 67, 70,
   72, 73, 74, 75, 76, 77, 78, 79, 80, 81,
   82, 84, 85, 86, 87, 88, 89, 90]

object_label[a[63]], verb_label[19]

('laptop', 'work_on_computer_instr')

In [7]:
import numpy as np

def co_appear(d, v):
    freq = d[v]
    d = dict({key : value / freq for key, value in d.items() if key != v})
    return d

for sbj in hois_dict.keys():
    for obj in hois_dict[sbj].keys():
        for verb in hois_dict[sbj][obj].keys():
            d = hois_dict[sbj][obj][verb]
            hois_dict[sbj][obj][verb] = co_appear(d, verb)

In [8]:
def come_out_together(s, o, v):
    dist = hois_dict[s][o][v]
    print("Subject : {} Object : {} Verb : {}".format(object_label[s], object_label[o], verb_label[v]))
    for k, v in dist.items():
        print("Verb : {} Score : {:.3f}".format(verb_label[k], v))
    print()
    
come_out_together(1, 2, 3)
come_out_together(1, 4, 0)

Subject : person Object : bicycle Verb : ride_instr
Verb : hold_obj Score : 0.522
Verb : sit_instr Score : 0.776
Verb : look_obj Score : 0.060

Subject : person Object : motorcycle Verb : hold_obj
Verb : sit_instr Score : 0.926
Verb : ride_instr Score : 0.971
Verb : look_obj Score : 0.029



In [9]:
import copy

def come_out_threshold(hois, threshold=0.7):
    d = copy.deepcopy(hois)
    for sbj in d.keys():
        for obj in d[sbj].keys():
            for verb_a in d[sbj][obj].keys():
                over_threshold = list()
                for verb_b in d[sbj][obj][verb_a].keys():
                    if d[sbj][obj][verb_a][verb_b] >= threshold:
                        over_threshold.append(verb_b)
                d[sbj][obj][verb_a] = over_threshold
    return d
filtered = come_out_threshold(hois_dict)
filtered[1][3][4]

[1]

In [10]:
import pickle
 
save_data = filtered
with open("/home/intern/seokwon/qpic/coappear.pickle", "wb") as file:
    pickle.dump(save_data, file)

In [11]:
for sample in train_anno:
    
    hoi_dict = dict()
    
    objs = sample['annotations']
    hois = sample['hoi_annotation']
    
    for hoi in hois:
        subject_class = objs[hoi['subject_id']]['category_id']
        object_class = objs[hoi['object_id']]['category_id'] if hoi['object_id'] != -1 else 0
        hoi_class = hoi['category_id']
        
        if subject_class not in hoi_dict:
            hoi_dict[subject_class] = dict()
        if object_class not in hoi_dict[subject_class]:
            hoi_dict[subject_class][object_class] = dict()
        if hoi_class not in hoi_dict[subject_class][object_class]:
            hoi_dict[subject_class][object_class][hoi_class] = 1
        else:
            hoi_dict[subject_class][object_class][hoi_class] += 1
    if 1 in hoi_dict:
        if 2 in hoi_dict[1]:
            if 2 in hoi_dict[1][2] and 3 in hoi_dict[1][2]:
                if hoi_dict[1][2][2] != hoi_dict[1][2][3]:
                    print(sample['file_name'])

COCO_train2014_000000387124.jpg
COCO_train2014_000000306800.jpg
COCO_train2014_000000029525.jpg
COCO_train2014_000000218546.jpg
COCO_train2014_000000099046.jpg
COCO_train2014_000000231524.jpg
